In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b7' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 400
start_epoch = 0
train_batch = 4
test_batch = 256
lr = 0.04
schedule = [75, 150, 225]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b7/100shot' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Image
ex_size = (140, 140)
size = (128, 128)

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# cutmix
cm_prob = 0.01
cm_prob_init = 0.99
cm_prob_low = 0.01
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0
best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(data_dir, '100_shot')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(ex_size),
    transforms.RandomCrop(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])

val_aug = transforms.Compose([
    transforms.Resize(ex_size),
    transforms.CenterCrop(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
model = EfficientNet.from_name(model_name, num_classes=num_classes)
#                               override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [11]:
model.named_parameters

<bound method Module.named_parameters of EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 64, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 64, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity(

In [12]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 63.79M


# Loss

In [13]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [14]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Train AUROC.', 'Valid AUROC.'])

# Train

In [15]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
#         auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
#         arc.update(auroc, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [16]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('{batch}/{size} | Loss:{loss:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
         batch=batch_idx+1, size=len(val_loader), loss=losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [17]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [18]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, train_auroc, test_auroc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 400] LR: 0.040000
50/50 | Loss:10.1488 | top1:52.0000 | AUROC:0.0000
31/31 | Loss:18.2650 | top1:50.0000 | AUROC:0.5150

Epoch: [2 | 400] LR: 0.068000
50/50 | Loss:17.6463 | top1:52.0000 | AUROC:0.0000
31/31 | Loss:10.0804 | top1:50.0000 | AUROC:0.5104

Epoch: [3 | 400] LR: 0.096000
50/50 | Loss:11.3045 | top1:49.5000 | AUROC:0.0000
31/31 | Loss:6.2548 | top1:50.0000 | AUROC:0.4716

Epoch: [4 | 400] LR: 0.124000
50/50 | Loss:7.0780 | top1:43.5000 | AUROC:0.0000
31/31 | Loss:5.3753 | top1:50.0000 | AUROC:0.4587

Epoch: [5 | 400] LR: 0.152000
50/50 | Loss:7.8161 | top1:45.5000 | AUROC:0.0000
31/31 | Loss:6.0753 | top1:50.0000 | AUROC:0.5085

Epoch: [6 | 400] LR: 0.180000
50/50 | Loss:10.4310 | top1:49.5000 | AUROC:0.0000
31/31 | Loss:7.1525 | top1:50.0000 | AUROC:0.5183

Epoch: [7 | 400] LR: 0.208000
50/50 | Loss:21.4719 | top1:50.0000 | AUROC:0.0000
31/31 | Loss:2.3592 | top1:50.0000 | AUROC:0.4746

Epoch: [8 | 400] LR: 0.236000
50/50 | Loss:31.9497 | top1:51.5000 | AUROC:0


Epoch: [63 | 400] LR: 0.307335
50/50 | Loss:1.3437 | top1:51.5000 | AUROC:0.0000
31/31 | Loss:0.8280 | top1:50.0000 | AUROC:0.5190

Epoch: [64 | 400] LR: 0.306841
50/50 | Loss:1.0495 | top1:50.0000 | AUROC:0.0000
31/31 | Loss:0.9802 | top1:50.0000 | AUROC:0.5068

Epoch: [65 | 400] LR: 0.306337
50/50 | Loss:1.1566 | top1:51.0000 | AUROC:0.0000
31/31 | Loss:1.1383 | top1:50.0000 | AUROC:0.4988

Epoch: [66 | 400] LR: 0.305825
50/50 | Loss:1.4619 | top1:44.0000 | AUROC:0.0000
31/31 | Loss:1.1535 | top1:50.0000 | AUROC:0.5059

Epoch: [67 | 400] LR: 0.305303
50/50 | Loss:2.1896 | top1:48.5000 | AUROC:0.0000
31/31 | Loss:0.8108 | top1:50.0000 | AUROC:0.4722

Epoch: [68 | 400] LR: 0.304772
50/50 | Loss:1.2716 | top1:51.5000 | AUROC:0.0000
31/31 | Loss:0.9357 | top1:50.0000 | AUROC:0.4522

Epoch: [69 | 400] LR: 0.304233
50/50 | Loss:1.5071 | top1:44.5000 | AUROC:0.0000
31/31 | Loss:0.7014 | top1:50.0000 | AUROC:0.5047

Epoch: [70 | 400] LR: 0.303684
50/50 | Loss:1.3790 | top1:55.0000 | AUROC:0


Epoch: [125 | 400] LR: 0.026102
50/50 | Loss:0.7980 | top1:52.5000 | AUROC:0.0000
31/31 | Loss:0.7545 | top1:50.0000 | AUROC:0.4756

Epoch: [126 | 400] LR: 0.026004
50/50 | Loss:0.8624 | top1:49.5000 | AUROC:0.0000
31/31 | Loss:0.7102 | top1:50.0000 | AUROC:0.4779

Epoch: [127 | 400] LR: 0.025906
50/50 | Loss:0.8132 | top1:50.0000 | AUROC:0.0000
31/31 | Loss:0.7567 | top1:50.0000 | AUROC:0.4756

Epoch: [128 | 400] LR: 0.025807
50/50 | Loss:0.8526 | top1:48.0000 | AUROC:0.0000
31/31 | Loss:0.7261 | top1:50.0000 | AUROC:0.4823

Epoch: [129 | 400] LR: 0.025707
50/50 | Loss:0.7820 | top1:52.0000 | AUROC:0.0000
31/31 | Loss:0.6938 | top1:50.0000 | AUROC:0.4828

Epoch: [130 | 400] LR: 0.025607
50/50 | Loss:0.8653 | top1:49.5000 | AUROC:0.0000
31/31 | Loss:0.7318 | top1:50.0000 | AUROC:0.4837

Epoch: [131 | 400] LR: 0.025506
50/50 | Loss:0.7907 | top1:53.5000 | AUROC:0.0000
31/31 | Loss:0.6993 | top1:50.0000 | AUROC:0.4842

Epoch: [132 | 400] LR: 0.025405
50/50 | Loss:0.7940 | top1:53.5000 |

31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4718

Epoch: [187 | 400] LR: 0.001912
50/50 | Loss:0.7868 | top1:51.0000 | AUROC:0.0000
31/31 | Loss:0.6935 | top1:50.0000 | AUROC:0.4724

Epoch: [188 | 400] LR: 0.001900
50/50 | Loss:0.7802 | top1:49.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4732

Epoch: [189 | 400] LR: 0.001887
50/50 | Loss:0.7885 | top1:47.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4741

Epoch: [190 | 400] LR: 0.001875
50/50 | Loss:0.8018 | top1:50.5000 | AUROC:0.0000
31/31 | Loss:0.6957 | top1:50.0000 | AUROC:0.4734

Epoch: [191 | 400] LR: 0.001863
50/50 | Loss:0.8179 | top1:47.5000 | AUROC:0.0000
31/31 | Loss:0.6956 | top1:50.0000 | AUROC:0.4715

Epoch: [192 | 400] LR: 0.001850
50/50 | Loss:0.8382 | top1:47.5000 | AUROC:0.0000
31/31 | Loss:0.7013 | top1:50.0000 | AUROC:0.4751

Epoch: [193 | 400] LR: 0.001838
50/50 | Loss:0.8281 | top1:48.5000 | AUROC:0.0000
31/31 | Loss:0.6934 | top1:50.0000 | AUROC:0.4737

Epoch: [194 | 400]

50/50 | Loss:0.7709 | top1:49.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:49.8462 | AUROC:0.4687

Epoch: [249 | 400] LR: 0.000114
50/50 | Loss:0.8390 | top1:48.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:49.3462 | AUROC:0.4691

Epoch: [250 | 400] LR: 0.000113
50/50 | Loss:0.7835 | top1:52.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4685

Epoch: [251 | 400] LR: 0.000112
50/50 | Loss:0.8263 | top1:42.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:49.8974 | AUROC:0.4693

Epoch: [252 | 400] LR: 0.000111
50/50 | Loss:0.7339 | top1:54.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:49.7436 | AUROC:0.4698

Epoch: [253 | 400] LR: 0.000109
50/50 | Loss:0.8147 | top1:51.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4699

Epoch: [254 | 400] LR: 0.000108
50/50 | Loss:0.7283 | top1:55.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4685

Epoch: [255 | 400] LR: 0.000107
50/50 | Loss:0.7722 | top1:51.0000 | AUROC:0.0000
31/31 | Loss:0.6932


Epoch: [310 | 400] LR: 0.000049
50/50 | Loss:0.7616 | top1:51.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4690

Epoch: [311 | 400] LR: 0.000048
50/50 | Loss:0.8327 | top1:49.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4691

Epoch: [312 | 400] LR: 0.000047
50/50 | Loss:0.8513 | top1:45.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4684

Epoch: [313 | 400] LR: 0.000046
50/50 | Loss:0.8023 | top1:46.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4692

Epoch: [314 | 400] LR: 0.000045
50/50 | Loss:0.8155 | top1:46.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4693

Epoch: [315 | 400] LR: 0.000044
50/50 | Loss:0.7749 | top1:48.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4681

Epoch: [316 | 400] LR: 0.000043
50/50 | Loss:0.8222 | top1:46.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4691

Epoch: [317 | 400] LR: 0.000043
50/50 | Loss:0.7974 | top1:51.5000 |

31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4687

Epoch: [372 | 400] LR: 0.000008
50/50 | Loss:0.8612 | top1:48.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4692

Epoch: [373 | 400] LR: 0.000007
50/50 | Loss:0.7703 | top1:56.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4687

Epoch: [374 | 400] LR: 0.000007
50/50 | Loss:0.8352 | top1:43.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4691

Epoch: [375 | 400] LR: 0.000007
50/50 | Loss:0.7275 | top1:55.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4700

Epoch: [376 | 400] LR: 0.000006
50/50 | Loss:0.8317 | top1:46.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4690

Epoch: [377 | 400] LR: 0.000006
50/50 | Loss:0.7978 | top1:49.0000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4685

Epoch: [378 | 400] LR: 0.000006
50/50 | Loss:0.8440 | top1:47.5000 | AUROC:0.0000
31/31 | Loss:0.6932 | top1:50.0000 | AUROC:0.4695

Epoch: [379 | 400]